# Instalar dependências

In [ ]:
!pip install requests beautifulsoup4 lxml pandas python-dateutil


# Banco de dados (SQLite)

In [ ]:
import sqlite3
from pathlib import Path

DB_PATH = Path("/content/jornalopcao_reportagens.db")

CREATE_TABLE_JO = """
CREATE TABLE IF NOT EXISTS jornalopcao_reportagens (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    subtitle TEXT,
    url TEXT NOT NULL UNIQUE,
    section TEXT,
    city TEXT,
    author TEXT,
    published_at TEXT,
    updated_at TEXT,
    text TEXT,
    tags TEXT,
    raw_html TEXT,
    scraped_at TEXT NOT NULL
);
"""

def get_conn():
    DB_PATH.parent.mkdir(parents=True, exist_ok=True)
    conn = sqlite3.connect(DB_PATH)
    conn.execute("PRAGMA journal_mode=WAL;")
    conn.execute("PRAGMA foreign_keys = ON;")
    return conn

def init_db_jornalopcao():
    conn = get_conn()
    with conn:
        conn.execute(CREATE_TABLE_JO)
    conn.close()

def insert_jornalopcao(conn, record: dict):
    """
    record deve ter as chaves:
      title, subtitle, url, section, city, author,
      published_at, updated_at, text, tags, raw_html, scraped_at
    """
    sql = """
    INSERT OR IGNORE INTO jornalopcao_reportagens
    (title, subtitle, url, section, city, author,
     published_at, updated_at, text, tags, raw_html, scraped_at)
    VALUES (:title, :subtitle, :url, :section, :city, :author,
            :published_at, :updated_at, :text, :tags, :raw_html, :scraped_at)
    """
    with conn:
        conn.execute(sql, record)


# Helpers de HTTP + parse de data

In [ ]:
import requests
from bs4 import BeautifulSoup
from dateutil import parser as dateparser
from datetime import datetime, timedelta
import time
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; JornalOpcaoScraper/1.0; +https://seu-email-ou-site)"
}

MONTHS_PT = [
    "janeiro", "fevereiro", "março", "marco", "abril", "maio",
    "junho", "julho", "agosto", "setembro", "outubro", "novembro", "dezembro"
]

def get_soup(url: str) -> BeautifulSoup:
    resp = requests.get(url, headers=HEADERS, timeout=10)
    resp.raise_for_status()
    return BeautifulSoup(resp.text, "lxml")

def parse_datetime(text: str):
    if not text:
        return None
    try:
        return dateparser.parse(text)
    except Exception:
        return None

def extract_date_text(soup: BeautifulSoup):
    """
    Procura algo tipo '06 dezembro 2025 às 21h21'
    dentro da matéria.
    """
    for s in soup.stripped_strings:
        txt = s.strip()
        lower = txt.lower()
        if any(m in lower for m in MONTHS_PT) and any(ch.isdigit() for ch in lower):
            return txt
    return None

from bs4 import BeautifulSoup
from bs4.element import NavigableString
import re

def extract_author_jornalopcao(soup: BeautifulSoup):
    """
    Extrai o autor numa página de reportagem do Jornal Opção.

    Estratégia:
    1) Tenta seletores "óbvios" (rel='author', classes, etc)
    2) Se não achar:
       - pega o <h1> (título)
       - varre os elementos SEGUINTES ao <h1>
       - o primeiro texto 'tipo nome' ANTES da data é o autor
    """

    # 1) seletores comuns (caso mudem o tema no futuro)
    selector_candidates = [
        'a[rel="author"]',
        'span[rel="author"]',
        '[itemprop="author"]',
        '.author',
        '.post-author',
        '.entry-author',
        '.single__author',
        '.meta-author',
    ]
    for sel in selector_candidates:
        el = soup.select_one(sel)
        if el:
            txt = el.get_text(strip=True)
            if txt:
                return txt

    # 2) pegar data como âncora de parada
    date_txt = extract_date_text(soup)

    # 3) pegar <h1> (título da matéria)
    title_tag = soup.find("h1")
    if not title_tag:
        return None

    # 4) varrer elementos após o <h1> até chegar na data
    for el in title_tag.next_elements:
        # Se for a própria string da data -> paramos
        if isinstance(el, NavigableString):
            txt = el.strip()
            if not txt:
                continue

            # se esse texto é igual à data, ou contém mês + número, paramos
            lower = txt.lower()
            if (date_txt and date_txt.strip() == txt.strip()) or (
                any(m in lower for m in MONTHS_PT) and any(ch.isdigit() for ch in lower)
            ):
                break

            # se for algo tipo "COMPARTILHAR" / "RELACIONADAS", paramos a busca
            if any(p in lower for p in ["compartilhar", "relacionadas"]):
                break

            # ignora coisas com número (ex: "49 ANOS")
            if any(ch.isdigit() for ch in txt):
                continue

            # ignora palavras de navegação
            if lower in {"início", "inicio", "reportagens"}:
                continue

            # heurística simples de "parece nome"
            if 2 <= len(txt) <= 60:
                return txt

            continue

        # Se for uma tag (a, span, etc.)
        if hasattr(el, "get_text"):
            txt = el.get_text(strip=True)
            if not txt:
                continue

            lower = txt.lower()

            # se chegou na data, paramos
            if (date_txt and date_txt.strip() == txt.strip()) or (
                any(m in lower for m in MONTHS_PT) and any(ch.isdigit() for ch in lower)
            ):
                break

            # parar se bater num bloco de compartilhamento
            if any(p in lower for p in ["compartilhar", "relacionadas"]):
                break

            # pular texto com números (49 ANOS, etc.)
            if any(ch.isdigit() for ch in txt):
                continue

            if lower in {"início", "inicio", "reportagens"}:
                continue

            # aqui é o candidato a autor: primeiro texto curto, sem número, logo após o h1
            if 2 <= len(txt) <= 60:
                return txt

    return None


def extract_section(soup: BeautifulSoup):
    """
    Pega algo tipo 'Política', 'História', etc.
    Se não tiver nada confiável, volta 'Reportagens'.
    """
    # tentativa simples: olhar breadcrumbs
    strings = list(soup.stripped_strings)
    for i, s in enumerate(strings):
        if s.strip() == "Reportagens" and i > 0:
            prev = strings[i - 1].strip()
            if prev not in {"Início", "Reportagens", "Últimas notícias"}:
                return prev
    return "Reportagens"

def extract_tags(soup: BeautifulSoup):
    """
    Usa links /assunto/ como tags.
    """
    tags = []
    for a in soup.select("a[href*='/assunto/']"):
        txt = a.get_text(strip=True)
        if txt:
            tags.append(txt)
    # remove duplicados mantendo ordem
    return list(dict.fromkeys(tags))

def extract_city_from_text(paragraphs):
    """
    Igual G1: tenta encontrar 'Cidade - ' na primeira linha.
    Se não achar nada, retorna None.
    """
    if not paragraphs:
        return None
    first_line = paragraphs[0]
    if " - " in first_line:
        maybe_city = first_line.split(" - ")[0]
        if len(maybe_city.split()) <= 4:
            return maybe_city
    return None


from bs4 import BeautifulSoup

def extract_text_jornalopcao(soup: BeautifulSoup) -> str:
    """
    Extrai o texto principal da reportagem do Jornal Opção,
    tentando imitar o comportamento do G1 (só o corpo da matéria).
    """

    # 1) Tenta pegar o <article> (WordPress costuma usar isso)
    article = soup.find("article")

    # 2) Se não achar, tenta alguns contêineres comuns
    if not article:
        for cls in ["single__content", "entry-content", "post-content"]:
            article = soup.find("div", class_=cls)
            if article:
                break

    # 3) Se ainda não achar, faz um fallback pro site inteiro (melhor que nada)
    container = article if article else soup

    paragraphs = []
    for el in container.find_all(["p", "h2", "h3"], recursive=True):
        txt = el.get_text(" ", strip=True)
        if not txt:
            continue

        lower = txt.lower()

        # Cortar na parte de "leia também" / rodapé
        if "leia também" in lower:
            break
        if "49 anos" in lower and "jornal opção" in lower:
            break

        # Ignorar coisas de navegação/compartilhamento
        if any(w in lower for w in ["compartilhar", "relacionadas", "colunas e blogs"]):
            continue

        paragraphs.append(txt)

    # Remover duplicatas mantendo ordem
    if not paragraphs:
        return None

    # Usa "\n\n" pra ficar legível tipo parágrafo
    text = "\n\n".join(dict.fromkeys(paragraphs))
    return text



# Listagem de reportagens por página

In [ ]:
def fetch_jornalopcao_article(url: str) -> dict:
    """
    Baixa e extrai dados de uma reportagem do Jornal Opção
    no mesmo formato da tabela do G1.
    """
    print(f"[ARTICLE] Baixando: {url}")
    soup = get_soup(url)

    # Título
    title_tag = soup.find("h1")
    title = title_tag.get_text(strip=True) if title_tag else None

    # Não parece ter subtítulo separado na maioria das matérias;
    # se quiser depois, dá pra tentar pegar o primeiro <h2> ou um <p> destacado.
    subtitle = None

    # Autor
    author = extract_author_jornalopcao(soup)

    # Data
    published_raw = extract_date_text(soup)
    published_iso = None
    dt = parse_datetime(published_raw) if published_raw else None
    if dt is not None:
        published_iso = dt.isoformat()

    # Normalmente não há data de atualização explícita
    updated_at = None

    # Seção
    section = extract_section(soup)

    # Corpo do texto: todos os <p>
    paragraphs = [p.get_text(" ", strip=True) for p in soup.find_all("p")]
    text = extract_text_jornalopcao(soup)

    # Cidade (heurística igual G1)
    city = extract_city_from_text(paragraphs)

    # Tags
    tags_list = extract_tags(soup)
    tags_str = json.dumps(tags_list, ensure_ascii=False) if tags_list else None

    raw_html = str(soup)
    scraped_at = datetime.utcnow().isoformat()

    return {
        "title": title,
        "subtitle": subtitle,
        "url": url,
        "section": section,
        "city": city,
        "author": author,
        "published_at": published_iso or published_raw,
        "updated_at": updated_at,
        "text": text,
        "tags": tags_str,
        "raw_html": raw_html,
        "scraped_at": scraped_at,
    }


A categoria tem essa cara:

* https://www.jornalopcao.com.br/categoria/reportagens/

* https://www.jornalopcao.com.br/categoria/reportagens/page/2/

* https://www.jornalopcao.com.br/categoria/reportagens/page/3/

com as reportagens em forma de cards.

Vamos pegar os links que apontam para matérias (tipicamente /ultimas-noticias/... ou /reportagens/...).

In [ ]:
BASE_JO_URL = "https://www.jornalopcao.com.br/categoria/reportagens/"

def fetch_jornalopcao_listing(page: int):
    if page == 1:
        url = BASE_JO_URL
    else:
        url = f"{BASE_JO_URL}page/{page}/"

    print(f"[LIST] Página {page}: {url}")
    soup = get_soup(url)

    urls = set()

    for a in soup.select("a"):
        href = a.get("href") or ""
        if not href.startswith("https://www.jornalopcao.com.br/"):
            continue

        if "/categoria/" in href or "/assunto/" in href or "/page/" in href:
            continue

        if "/ultimas-noticias/" in href or "/reportagens/" in href:
            urls.add(href)

    urls = sorted(urls)
    print(f"[LIST] Encontradas {len(urls)} URLs de matérias na página {page}.")
    return urls


In [ ]:
def crawl_jornalopcao_reportagens(max_pages: int = 50, months: int = 6):
    """
    Percorre as páginas da categoria 'Reportagens' e salva apenas
    matérias dos últimos `months` meses (aprox), usando o mesmo
    esquema de colunas do G1.
    """
    init_db_jornalopcao()
    conn = get_conn()

    cutoff = datetime.now() - timedelta(days=30 * months)
    print(f"[INFO] Data de corte aproximada: {cutoff.date()} (últimos {months} meses)")

    seen = set()
    pages_only_old = 0

    for page in range(1, max_pages + 1):
        try:
            article_urls = fetch_jornalopcao_listing(page)
        except Exception as e:
            print(f"[STOP] Erro ao listar página {page}: {e}")
            break

        if not article_urls:
            print(f"[STOP] Página {page} veio sem matérias, encerrando.")
            break

        page_has_newer = False

        for url in article_urls:
            if url in seen:
                continue
            seen.add(url)

            try:
                record = fetch_jornalopcao_article(url)
            except Exception as e:
                print(f"[ERROR] {url}: {e}")
                continue

            if not record.get("title") or not record.get("text"):
                print(f"[WARN] Sem título ou texto, pulando: {url}")
                continue

            # filtro de data
            pub_str = record.get("published_at")
            pub_dt = parse_datetime(pub_str) if pub_str else None

            if pub_dt is not None:
                if pub_dt.date() < cutoff.date():
                    print(
                        f"[SKIP old] {record['title'][:60]}... "
                        f"({pub_dt.date()}) < corte ({cutoff.date()})"
                    )
                    continue
                else:
                    page_has_newer = True
            else:
                # se não tem data parseável, eu salvo mesmo assim
                page_has_newer = True

            insert_jornalopcao(conn, record)
            print(f"[OK] {record['title'][:80]}")
            time.sleep(1)

        if not page_has_newer:
            pages_only_old += 1
            print(f"[INFO] Página {page} só tinha notícia velha.")
            if pages_only_old >= 3:
                print(
                    f"[STOP] {pages_only_old} páginas seguidas só com notícia < corte. Encerrando."
                )
                break
        else:
            pages_only_old = 0

        time.sleep(1)

    conn.close()
    print("[DONE] Crawl Jornal Opção finalizado.")


Mude aqui para a quantidade de páginas, ou o tanto de meses que você quer que ele pegue

In [ ]:
crawl_jornalopcao_reportagens(max_pages=20, months=6)


[INFO] Data de corte aproximada: 2025-06-16 (últimos 6 meses)
[LIST] Página 1: https://www.jornalopcao.com.br/categoria/reportagens/
[LIST] Encontradas 10 URLs de matérias na página 1.
[ARTICLE] Baixando: https://www.jornalopcao.com.br/reportagens/construida-por-missionario-alemao-casa-mais-antiga-de-goiania-completa-100-anos-763925/


/tmp/ipython-input-3419327968.py:45: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  scraped_at = datetime.utcnow().isoformat()


[OK] Construída por missionário alemão, casa mais antiga de Goiânia completa 100 anos
[ARTICLE] Baixando: https://www.jornalopcao.com.br/reportagens/goias-na-encruzilhada-fiscal-como-jose-eliton-recebeu-o-estado-de-marconi-e-como-o-entregou-para-caiado-766995/
[OK] Goiás na encruzilhada fiscal: como José Eliton recebeu o Estado de Marconi e com
[ARTICLE] Baixando: https://www.jornalopcao.com.br/reportagens/pl-avanca-na-formacao-das-chapas-para-2026-em-goias-772230/
[OK] PL avança na formação das chapas para 2026 em Goiás
[ARTICLE] Baixando: https://www.jornalopcao.com.br/ultimas-noticias/autoestima-e-poder-mundial-o-exemplo-dos-estados-unidos-serve-para-a-china-772591/
[OK] Autoestima e poder mundial: o exemplo dos Estados Unidos serve para a China?
[ARTICLE] Baixando: https://www.jornalopcao.com.br/ultimas-noticias/conheca-5-empresas-que-estao-transformando-o-mercado-da-inteligencia-artificial-em-goias-763944/
[OK] Conheça 5 empresas que estão transformando o mercado da Inteligência A

In [ ]:
import pandas as pd

conn = get_conn()
df = pd.read_sql_query("""
    SELECT *
    FROM jornalopcao_reportagens
    ORDER BY id ASC
""", conn)
conn.close()

df


,id,title,subtitle,url,section,city,author,published_at,updated_at,text,tags,raw_html,scraped_at
0,1,"Construída por missionário alemão, casa mais a...",None,https://www.jornalopcao.com.br/reportagens/con...,Opção Play,None,Herbert Moraes,08 novembro 2025 às 21h00,None,"Dona Aparecida tem 62 anos e, desde pequena, p...",None,"<!DOCTYPE html>\n<html lang=""pt-BR"">\n<head>\n...",2025-12-13T20:17:07.277208
1,2,Goiás na encruzilhada fiscal: como José Eliton...,None,https://www.jornalopcao.com.br/reportagens/goi...,Opção Play,None,Cilas Gontijo,22 novembro 2025 às 21h00,None,O Governo de Goiás encerrou o exercício de 202...,"[""Contas"", ""Goiás"", ""marconi"", ""Perillo"", ""Zé ...","<!DOCTYPE html>\n<html lang=""pt-BR"">\n<head>\n...",2025-12-13T20:17:08.452268
2,3,PL avança na formação das chapas para 2026 em ...,None,https://www.jornalopcao.com.br/reportagens/pl-...,Opção Play,None,Cilas Gontijo,06 dezembro 2025 às 21h00,None,A direção do Partido Liberal (PL) em Goiás con...,"[""Goiás"", ""PL"", ""Pré-candidatos""]","<!DOCTYPE html>\n<html lang=""pt-BR"">\n<head>\n...",2025-12-13T20:17:09.654087
3,4,Autoestima e poder mundial: o exemplo dos Esta...,None,https://www.jornalopcao.com.br/ultimas-noticia...,Opção Play,None,Redação,06 dezembro 2025 às 21h21,None,J. C. Guimarães\n\nEspecial para o Jornal Opçã...,None,"<!DOCTYPE html>\n<html lang=""pt-BR"">\n<head>\n...",2025-12-13T20:17:10.916280
4,5,Conheça 5 empresas que estão transformando o m...,None,https://www.jornalopcao.com.br/ultimas-noticia...,Opção Play,None,Tathyane Melo,08 novembro 2025 às 21h00,None,*Colaboração de Amanda Costa\n\nLonge do merca...,"[""Goiânia"", ""Goiás"", ""IA"", ""Inovação"", ""Inteli...","<!DOCTYPE html>\n<html lang=""pt-BR"">\n<head>\n...",2025-12-13T20:17:12.420623
...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,260,Veja quem são os pré-candidatos a prefeito de ...,None,https://www.jornalopcao.com.br/reportagens/vej...,Opção Play,None,Cilas Gontijo,09 junho 2024 às 00h00,None,"Das cinco maiores cidades do sudoeste goiano, ...","[""Cidades"", ""Goiano"", ""Pré-candidatos"", ""Prefe...","<!DOCTYPE html>\n<html lang=""pt-BR"">\n<head>\n...",2025-12-13T20:23:44.515589
260,261,Doação da antiga sede da Alego para o TCM-GO v...,None,https://www.jornalopcao.com.br/ultimas-noticia...,Opção Play,None,Cilas Gontijo,12 maio 2024 às 00h00,None,Os membros fundadores da Associação dos Protet...,"[""Ação"", ""Alego"", ""Justiça"", ""Leonardo Rizzo"",...","<!DOCTYPE html>\n<html lang=""pt-BR"">\n<head>\n...",2025-12-13T20:23:45.790617
261,262,"Em 31 anos, Goiás registrou 47 enchentes e inu...",None,https://www.jornalopcao.com.br/ultimas-noticia...,Opção Play,None,Raphael Bezerra,11 maio 2024 às 09h35,None,Eventos climáticos como as inundações em Alago...,"[""Chuva"", ""desabrigado"", ""Enchente"", ""Goiás"", ...","<!DOCTYPE html>\n<html lang=""pt-BR"">\n<head>\n...",2025-12-13T20:23:47.185834
262,263,"Em Goiânia, líderes evangélicos se dividem em ...",None,https://www.jornalopcao.com.br/ultimas-noticia...,Opção Play,None,Ton Paulo,19 maio 2024 às 00h00,None,“Nunca se soube de um cenário eleitoral tão ab...,None,"<!DOCTYPE html>\n<html lang=""pt-BR"">\n<head>\n...",2025-12-13T20:23:48.758654


In [ ]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            264 non-null    int64 
 1   title         264 non-null    object
 2   subtitle      0 non-null      object
 3   url           264 non-null    object
 4   section       264 non-null    object
 5   city          0 non-null      object
 6   author        264 non-null    object
 7   published_at  264 non-null    object
 8   updated_at    0 non-null      object
 9   text          264 non-null    object
 10  tags          184 non-null    object
 11  raw_html      264 non-null    object
 12  scraped_at    264 non-null    object
dtypes: int64(1), object(12)
memory usage: 145.8 MB


In [ ]:
df = df.drop(columns=['raw_html'])
display(df.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            264 non-null    int64 
 1   title         264 non-null    object
 2   subtitle      0 non-null      object
 3   url           264 non-null    object
 4   section       264 non-null    object
 5   city          0 non-null      object
 6   author        264 non-null    object
 7   published_at  264 non-null    object
 8   updated_at    0 non-null      object
 9   text          264 non-null    object
 10  tags          184 non-null    object
 11  scraped_at    264 non-null    object
dtypes: int64(1), object(11)
memory usage: 9.6 MB


None

In [ ]:
df.to_csv("jornal_opcao_noticias.csv")